# NORTHWIND EXPLORATORY DATA ANALYSIS USING SQL

At this JupyterNotebook I'm going to answer different business questions based on the Northwind Database, this DB was create on PostgreSQL so, first of all, I have to connect to my local host and then launch the queries.

To make it possible I used the following libraries:
 * Pandas: To store the created dataframes
 * Pandas.io.sql: To read the SQL queries into dataframes
 * Psycog2: To connect the notebook to the PostgreSQL engine
 * Warnings: To make expections on warnings

**Business Questions:**
  * What is the total number of clients per country?
  * What is the total number of orders per country?
  * What is the average price per order per country?
  * What is the total invoice for each country?
  * What is the total profit for each country?
  * What are the 10 most ordered products?
  * What are the 10 most expensive products?
  * What are the 10 products with the highest invoice?
  * What are the 10 products with the highest unit profit? 
  * What are the 10 most profitable products?

## Set up

### Import the libraries

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps
import warnings

### Connect to PostgreSQL

In [2]:
conn2 = ps.connect(dbname = "northwind", #Database name
                  user = "user", #Postgres username
                  password = "password", #Postgres password
                  host = "host_ip", #Server IP
                  port = "5432") #Virtual point of connection

### Quit all the warnings

In [3]:
warnings.filterwarnings('ignore')

## Exploratory Data Analysis

### Question 1 - What is the total number of clients per country?

In [4]:
# SQL Query

sql = """

SELECT country as "Countries", COUNT(*) as "Number of Clients"
        FROM customers
        GROUP BY country
        ORDER BY COUNT(*) DESC;
    
"""

In [5]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [6]:
# Visualize the created dataframe

df.head(100)

,Countries,Number of Clients
0,USA,13
1,Germany,11
2,France,11
3,Brazil,9
4,UK,7
5,Mexico,5
6,Spain,5
7,Venezuela,4
8,Argentina,3
9,Italy,3


### Question 2 - What is the total number of orders per country?

In [7]:
# SQL Query

sql = """

SELECT 

	c.country AS "Country",
  	COUNT(c.country) AS "Number of Orders"
	
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN customers c ON c.customer_id = orders.customer_id
	
	GROUP BY c.country
	ORDER BY "Number of Orders" DESC;
    
"""

In [8]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [9]:
# Visualize the created dataframe

df.head(100)

,Country,Number of Orders
0,USA,352
1,Germany,328
2,Brazil,203
3,France,184
4,UK,135
5,Austria,125
6,Venezuela,118
7,Sweden,97
8,Canada,75
9,Mexico,72


### Question 3 - What is the average price per order per country?

In [10]:
# SQL Query

sql = """

SELECT 

	c.country AS "Country",
  
	COUNT(c.country) AS "Number of Orders",
	SUM (o_d.quantity * o_d.unit_price * (1 - o_d.discount))/ CAST(COUNT(c.country) AS float8) AS "Average Order Income"
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN customers c ON c.customer_id = orders.customer_id
	
	GROUP BY c.country
	ORDER BY "Average Order Income" DESC;
    
"""

In [11]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [12]:
# Visualize the created dataframe

df.head(100)

,Country,Number of Orders,Average Order Income
0,Austria,125,1024.030705
1,Ireland,55,908.725547
2,Denmark,46,710.022228
3,Germany,328,702.087297
4,USA,352,697.683552
5,Canada,75,669.283868
6,Switzerland,52,609.474211
7,Belgium,56,604.015273
8,Sweden,97,561.805566
9,Brazil,203,526.727963


### Question 4 - What is the total invoice for each country?

In [13]:
# SQL Query

sql = """

SELECT 

	c.country AS "Countries",
  	SUM (o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS "Total Sales by Country"

	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN customers c ON c.customer_id = orders.customer_id
	
	GROUP BY c.country
	ORDER BY "Total Sales by Country" DESC;
    
"""

In [14]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [15]:
# Visualize the created dataframe

df.head(100)

,Countries,Total Sales by Country
0,USA,245584.610302
1,Germany,230284.633254
2,Austria,128003.838159
3,Brazil,106925.776404
4,France,81358.322530
5,UK,58971.309912
6,Venezuela,56810.628759
7,Sweden,54495.139921
8,Canada,50196.290101
9,Ireland,49979.905081


### Question 5 - What is the total profit for each country?

In [37]:
# SQL Query

sql = """

SELECT 

	c.country AS "Countries",
	
	SUM ((o_d.quantity * p.unit_price) - (o_d.quantity * o_d.unit_price * (1 - o_d.discount))) AS "Total Profit"
	
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN customers c ON c.customer_id = orders.customer_id
	INNER JOIN products p ON p.product_id = o_d.product_id
	
	GROUP BY c.country 
	ORDER BY "Total Profit" DESC;
    
"""

In [38]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [39]:
# Visualize the created dataframe

df.head(100)

,Countries,Total Profit
0,USA,35003.769685
1,Germany,27271.036347
2,Austria,21981.251800
3,Brazil,16808.953295
4,France,12220.837307
5,Canada,11703.859815
6,Ireland,10418.004972
7,Venezuela,8786.560963
8,Sweden,7338.559940
9,Denmark,6378.047377


### Question 6 - What are the 10 most ordered products?

In [21]:
# SQL Query

sql = """

SELECT DISTINCT

	p.product_name AS "Products",
	o_d.quantity AS "Times Ordered"
		
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN products p ON p.product_id = o_d.product_id
	
	GROUP BY p.product_name, o_d.quantity
	ORDER BY "Times Ordered" DESC
	LIMIT 10;
    
"""

In [22]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [23]:
# Visualize the created dataframe

df.head(100)

,Products,Times Ordered
0,Chartreuse verte,130
1,Wimmers gute Semmelknödel,130
2,Manjimup Dried Apples,120
3,Sirop d'érable,120
4,Rhönbräu Klosterbier,120
5,Perth Pasties,120
6,Jack's New England Clam Chowder,120
7,Schoggi Schokolade,120
8,Pâté chinois,120
9,Raclette Courdavault,110


### Question 7 - What are the 10 most expensive products?

In [24]:
# SQL Query

sql = """

SELECT DISTINCT

	p.product_name AS "Products",
	p.unit_price AS "Sale Unit Price"
		
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN products p ON p.product_id = o_d.product_id
	
	GROUP BY p.product_name, p.unit_price
	ORDER BY "Sale Unit Price" DESC
	LIMIT 10;
    
"""

In [25]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [26]:
# Visualize the created dataframe

df.head(100)

,Products,Sale Unit Price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
5,Raclette Courdavault,55.00
6,Manjimup Dried Apples,53.00
7,Tarte au sucre,49.30
8,Ipoh Coffee,46.00
9,Rössle Sauerkraut,45.60


### Question 8 - What are the 10 products with the highest invoice?

In [27]:
# SQL Query

sql = """

SELECT DISTINCT

	p.product_name AS "Products",
	SUM (o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS "Total Sales by Product"
		
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN products p ON p.product_id = o_d.product_id
	
	GROUP BY p.product_name, p.unit_price
	ORDER BY "Total Sales by Product" DESC
	LIMIT 10;
    
"""

In [28]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [29]:
# Visualize the created dataframe

df.head(100)

,Products,Total Sales by Product
0,Côte de Blaye,141396.735627
1,Thüringer Rostbratwurst,80368.672439
2,Raclette Courdavault,71155.699909
3,Tarte au sucre,47234.969979
4,Camembert Pierrot,46825.480295
5,Gnocchi di nonna Alice,42593.059822
6,Manjimup Dried Apples,41819.650246
7,Alice Mutton,32698.380216
8,Carnarvon Tigers,29171.874963
9,Rössle Sauerkraut,25696.639789


### Question 9 - What are the 10 products with the highest profit per unit? 

In [30]:
# SQL Query

sql = """

SELECT DISTINCT

	p.product_name AS "Products",
	SUM (p.unit_price - (o_d.unit_price * (1 - o_d.discount))) AS "Unitary Benefits by Product"
		
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN products p ON p.product_id = o_d.product_id
	
	GROUP BY p.product_name, p.unit_price
	ORDER BY "Unitary Benefits by Product" DESC
	LIMIT 10;
    
"""

In [31]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [32]:
# Visualize the created dataframe

df.head(100)

,Products,Unitary Benefits by Product
0,Côte de Blaye,671.924981
1,Thüringer Rostbratwurst,492.993012
2,Raclette Courdavault,336.600003
3,Tarte au sucre,263.314968
4,Gnocchi di nonna Alice,242.440008
5,Carnarvon Tigers,205.000001
6,Camembert Pierrot,197.879992
7,Manjimup Dried Apples,195.039989
8,Alice Mutton,176.669992
9,Rössle Sauerkraut,173.379960


### Question 10 - What are the 10 most profitable products?

In [33]:
# SQL Query

sql = """

SELECT DISTINCT

	p.product_name AS "Products",
	SUM ((o_d.quantity * p.unit_price) - (o_d.quantity * o_d.unit_price * (1 - o_d.discount))) 
    AS "Total Benefits by Product"
		
	FROM orders
	
	INNER JOIN order_details o_d ON o_d.order_id = orders.order_id
	INNER JOIN products p ON p.product_id = o_d.product_id
	
	GROUP BY p.product_name, p.unit_price
	ORDER BY "Total Benefits by Product" DESC
	LIMIT 10;
    
"""

In [34]:
# Execute the query on the data base and save it into a dataframe

df = sqlio.read_sql_query(sql, conn2)

In [35]:
# Visualize the created dataframe

df.head(100)

,Products,Total Benefits by Product
0,Côte de Blaye,22763.764373
1,Thüringer Rostbratwurst,11978.668244
2,Raclette Courdavault,11124.300091
3,Camembert Pierrot,6792.519705
4,Tarte au sucre,6156.929195
5,Alice Mutton,5443.619784
6,Gnocchi di nonna Alice,5400.940178
7,Manjimup Dried Apples,5138.349754
8,Carnarvon Tigers,4515.625037
9,Pâté chinois,4245.599730


## Conclussions about the Analysis

Once this EDA has been carried out, a series of conclusions are drawn:

 * The countries with the highest number of clients are USA, Germany, France and Brazil. 
 * The countries with the highest number of orders are USA, Germany, Brazil and France. 
 * The countries with the highest average order income are Austria, Ireland, Denmark and Germany.
 * The countries with the highest total income are USA, Germany, Austria and Brazil.
 * The countries with the highest total profit are USA, Germany, Austria and Brazil.
 
According to the information extracted about the different countries and markets, it can be corroborated that the most developed and stable markets are USA, Germany and Brazil.

 * The most expensive product doubles the price of the previous product on the list.
 * The most expensive product, despite its high price, generates the higher income.
 * Surprisingly, the most expensive product is also the one with the highest profit, both on a unit and overall basis.

*Álvarez Redondo, Miguel 29/09/2022*              